In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip

--2022-09-11 18:11:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116104947 (111M) [application/x-httpd-php]
Saving to: ‘data.zip’

data.zip            100%[===================>] 110.73M  37.7MB/s    in 2.9s    

2022-09-11 18:11:14 (37.7 MB/s) - ‘data.zip’ saved [116104947/116104947]



In [ ]:
!unzip data.zip -d data

Archive:  data.zip
  inflating: data/BitcoinHeistData.csv  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gzip, pickle


In [ ]:
df=pd.read_csv('/content/data/BitcoinHeistData.csv')

In [ ]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [ ]:
lb=LabelEncoder()
df['address']=lb.fit_transform(df['address'])
df['label']=lb.fit_transform(df['label'])

In [ ]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,23,2017,11,18,0.008333,1,0,2,100050000.0,26
1,128,2016,132,44,0.000244,1,0,1,100000000.0,27
2,169,2016,246,0,1.000000,1,0,2,200000000.0,26
3,217,2016,322,72,0.003906,1,0,2,71200000.0,26
4,293,2016,238,144,0.072848,456,0,1,200000000.0,27


In [ ]:
new_df=df.sample(frac=1)

In [ ]:
new_df.loc[new_df['address'] == 128]

,address,year,day,length,weight,count,looped,neighbors,income,label
1,128,2016,132,44,0.000244,1,0,1,100000000.0,27


In [ ]:
new_df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
1427894,1374599,2014,302,144,0.000081,1711,1658,2,1.004951e+08,28
2398026,1347537,2017,177,144,2.450266,8998,8998,2,1.000100e+08,28
1898050,236400,2016,42,20,0.062500,1,0,2,9.281077e+09,28
352194,1003124,2011,321,0,0.500000,1,0,1,7.977239e+08,28
49193,2291451,2011,14,20,0.511719,3,0,3,1.671000e+09,28


In [ ]:
Y=new_df['label']
X=new_df.drop('label',axis=1)

In [ ]:
X.head()

,address,year,day,length,weight,count,looped,neighbors,income
1427894,1374599,2014,302,144,0.000081,1711,1658,2,1.004951e+08
2398026,1347537,2017,177,144,2.450266,8998,8998,2,1.000100e+08
1898050,236400,2016,42,20,0.062500,1,0,2,9.281077e+09
352194,1003124,2011,321,0,0.500000,1,0,1,7.977239e+08
49193,2291451,2011,14,20,0.511719,3,0,3,1.671000e+09


In [ ]:
Y.head()

1427894    28
2398026    28
1898050    28
352194     28
49193      28
Name: label, dtype: int64

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.5,shuffle=False)

In [ ]:
from joblib import Parallel, delayed,dump
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle

In [ ]:
model_rf=RandomForestClassifier(max_depth=2,random_state=42)
model_dt=DecisionTreeClassifier(splitter="random",max_depth=2,random_state=2,max_features="auto")
models=[]
models.append(model_rf)
models.append(model_dt)

In [ ]:
def train_model(X,y,model,count):
  print('Executing model number',count)
  output=model.fit(X,y)
  with gzip.open('./'+str(count)+'.pkl', 'wb') as ofp:
    pickle.dump(output, ofp)
  return output




In [ ]:
%%time

result = Parallel(n_jobs=2)(delayed(train_model)(X_train, y_train, models[i],i) for i in range(len(models)))

CPU times: user 511 ms, sys: 246 ms, total: 757 ms
Wall time: 1min 25s


LOADING MODEL IN PARALLLEL AND MAKING PREDICTION ON TEST SET

In [ ]:
def test_model(X_test,y_test,model,model_count):

  filename='/content/'+str(model_count)+'.pkl'
  with gzip.open(filename, 'rb') as ifp:
    loaded_model=pickle.load(ifp)
    res=loaded_model.predict(X_test)
  return res


In [ ]:
%%time

result = Parallel(n_jobs=2)(delayed(test_model)(X_test, y_test, result[i],i) for i in range(len(models)))

CPU times: user 376 ms, sys: 119 ms, total: 495 ms
Wall time: 23.4 s


In [ ]:
result_df=pd.DataFrame({'RF':result[0],'DT':result[1]})

In [ ]:
result_df['Average']=result_df.mean(axis=1)

In [ ]:
result_df

,RF,DT,Average
0,28,28,28.0
1,28,28,28.0
2,28,28,28.0
3,28,28,28.0
4,28,28,28.0
...,...,...,...
1458344,28,28,28.0
1458345,28,28,28.0
1458346,28,28,28.0
1458347,28,28,28.0
